# Titanic Survival Prediction using FEDOT and LLM

This notebook demonstrates the process of analyzing the Titanic dataset and predicting passenger survival using the FEDOT framework enhanced with Large Language Models (LLM).

https://www.kaggle.com/competitions/titanic

In [1]:
import sys, os

module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if module_path not in sys.path:
    sys.path.insert(0, module_path)

from fedotllm.main import FedotAI
from fedotllm.output import JupyterOutput
from fedotllm.llm import AIInference
from examples.kaggle import download_from_kaggle, submit_to_kaggle

competition_name = "titanic"
dataset_path = os.path.join(os.getcwd(), "competition")
download_from_kaggle(competition_name=competition_name, save_path=dataset_path)

/Users/aleksejlapin/Work/STABLE-FedotLLM/.venv/lib/python3.10/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Dataset downloaded and extracted to /Users/aleksejlapin/Work/STABLE-FedotLLM/examples/titanic2/competition


In [2]:
description = """
Построй модель AutoML. Задача «Титаник - машинное обучение на основе катастрофы» на Kaggle заключается в предсказании того, выжил ли пассажир после затопления «Титаника», используя предоставленные наборы данных. Набор данных train.csv включает 891 пассажира с их статусом выживания (ground truth), а test.csv содержит аналогичную информацию, но без статуса выживания. Участники строят модель машинного обучения, используя закономерности в обучающих данных, чтобы предсказать результаты выживания в тестовых данных. Материалы должны быть представлены в формате CSV, содержащем ровно 418 строк и два столбца: PassengerId и Survived (бинарные прогнозы: 1 - выжил, 0 - умер). Метрикой оценки является точность, которая рассчитывается как процент правильных предсказаний.
"""

In [3]:
import shutil
output_path = os.path.join(os.getcwd(), 'output')
if os.path.exists(output_path):
    shutil.rmtree(output_path)
os.makedirs(output_path, exist_ok=True)

inference = AIInference()
    
fedot_ai = FedotAI(
        task_path=dataset_path,
        inference=inference,
        workspace=output_path,
        handlers=JupyterOutput().subscribe
    )
async for _ in fedot_ai.ask(message=description):
    continue

================== HumanMessage ==================

Создайте модель AutoML. Задача 'Titanic - Machine Learning from Disaster' на Kaggle заключается в предсказании, выжил ли пассажир после крушения "Титаника", используя предоставленные наборы данных. Набор данных train.csv включает 891 пассажира с их статусом выживания (истинные значения), в то время как test.csv содержит аналогичную информацию, но без статуса выживания. Участники создают модель машинного обучения, используя закономерности в обучающих данных, чтобы предсказать результаты выживания для тестовых данных. Отправляемые решения должны быть в формате CSV, содержать ровно 418 строк и две колонки: PassengerId и Survived (бинарные предсказания: 1 - выжил, 0 - умер). Метрикой оценки является точность (accuracy), которая рассчитывается как процент правильных предсказаний.

================== AutoMLAgent ===================

## Расшифровка выживаемости на «Титанике»: глубокое погружение в AutoML

Вы когда-нибудь задумывались, как искусственный интеллект решает исторические загадки? Недавно мы протестировали передовую модель AutoML, стремясь предсказать выживаемость на злополучном «Титанике». В этом отчете подробно описывается, как наш цифровой детектив собирал воедино улики из прошлого.

### 1. Обзор

*   **Проблема:** Задача заключалась в том, чтобы предсказать, выжил ли пассажир в катастрофе «Титаника». Мы использовали набор данных из 891 пассажира с известными исходами для обучения нашей модели, а затем применили эти знания для предсказания судьбы 418 неизвестных пассажиров. Конечная цель? Точно угадать «выжил» (1) или «погиб» (0) для каждого человека.
*   **Цель:** Цель нашей модели была проста: выступать в качестве сложного предиктора, просеивая информацию о пассажирах, такую как возраст, класс и порт посадки, чтобы определить, кто выжил, а кто погиб.

### 2. Предварительная обработка данных

Прежде чем наша модель смогла учиться, нам пришлось подготовить необработанные данные. Представьте это как уборку и организацию грязной комнаты, прежде чем вы сможете в ней заниматься.

*   **Очистка:** Мы удалили столбцы, которые были либо уникальными идентификаторами (например, `PassengerId`), либо слишком сложными для прямого использования (например, `Name`, `Ticket`, номера `Cabin`), либо в целом бесполезными для широкого распознавания образов. Это сфокусировало модель на действительно информативных характеристиках.
*   **Заполнение пробелов:** Наборы данных часто содержат недостающую информацию.
    *   Для числовых признаков, таких как `Age`, пропущенные значения были заменены с помощью **импутации по среднему**. Это означает, что если возраст пассажира был неизвестен, мы заменяли его средним возрастом всех остальных пассажиров.
    *   Для категориальных признаков (таких как порт `Embarked`) пропущенные записи были заполнены с использованием **импутации по наиболее частому значению**, заменяя их наиболее распространенным значением в этом столбце. Это обеспечивает единообразие во всем нашем наборе данных.
*   **Масштабирование для справедливости:** Хотя это не было явно закодировано как отдельный шаг в нашем пользовательском скрипте, фреймворк AutoML часто включает процесс, называемый **нормализацией** или **масштабированием**, внутри себя. Это гарантирует, что все признаки находятся в одном масштабе, что улучшает производительность модели. Например, процесс масштабирования преобразует значения, такие как «возраст» (который может варьироваться от 5 до 90 лет), в стандартизированный диапазон, скажем, от 0 до 1, предотвращая несправедливое доминирование признаков с большим числовым диапазоном в процессе обучения.

### 3. Сводка по конвейеру

Наша модель AutoML не просто выбрала один алгоритм; она построила сложный **ансамбль** или **стековый конвейер**. Представьте себе, что вы собираете команду мечты из опытных детективов, каждый из которых использует свои уникальные навыки для раскрытия сложного дела.

*   **Мозговой центр операции:**
    *   `depth: 3`, `length: 5`, `nodes: [logit, catboost, scaling, xgboost, lgbm]`
    *   Это указывает на многослойную структуру, где различные «узлы» (модели или шаги предварительной обработки) работают последовательно или параллельно, объединяя свои сильные стороны для получения более точных предсказаний. Окончательное предсказание часто представляет собой продуманную комбинацию того, за что «проголосовали» эти отдельные эксперты.

*   **Ключевые игроки и их роли:**

| Модель     | Параметры                                                                   | Объяснение                                                                                                                                                                                                                                                                                                                                                          |
| :-------- | :--------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **CatBoost** | `num_trees: 3000`, `learning_rate: 0.03`, `max_depth: 5`, `l2_leaf_reg: 0.01` | CatBoost — мощный алгоритм «градиентного бустинга», подобный команде слабых учеников, которые постепенно улучшаются, исправляя ошибки друг друга. Он был выбран за его исключительную способность напрямую обрабатывать **категориальные признаки** (такие как «Пол» или «Порт посадки») без необходимости сложных ручных преобразований, что часто приводит к более надежным моделям. |
| **Logit** | `{}`                                                                         | Сокращение от **Логистическая регрессия**, это фундаментальная статистическая модель. Это похоже на проведение прямой линии (или кривой в более высоких измерениях) для разделения двух групп (выжившие против невыживших). Ее простота часто обеспечивает хорошее базовое понимание взаимосвязей в данных.                                                                             |
| **XGBoost** | `n_jobs: 1`, `verbosity: 0`, `booster: 'gbtree'`, `tree_method: 'auto'`, `enable_categorical: True`, `use_eval_set: True`, `early_stopping_rounds: 30` | Еще один ведущий алгоритм «градиентного бустинга», похожий на CatBoost. XGBoost ценится за свою скорость и производительность. Его древовидная природа помогает ему улавливать сложные, нелинейные взаимосвязи в данных, что делает его очень эффективным для разнообразных наборов данных.                                                                                                    |
| **LightGBM** | `boosting_type: 'gbdt'`, `max_depth: -1`, `bagging_fraction: 0.85`, `extra_trees: False`, `enable_categorical: True`, `use_eval_set: True`, `early_stopping_rounds: 30`, `n_jobs: 1`, `verbose: -1` | Быстрый и эффективный фреймворк «градиентного бустинга», часто превосходящий другие древовидные методы на больших наборах данных. LightGBM оптимизирует процесс построения дерева, позволяя ему очень быстро обучаться, сохраняя при этом высокую точность, что делает его отличным для крупномасштабных приложений.                                                                                          |
| **Scaling** | `{}`                                                                         | Этот узел, обычно находящийся в начале конвейера, гарантирует, что все числовые признаки данных находятся в сопоставимом масштабе. Это предотвращает непреднамеренное доминирование признаков с большими числовыми диапазонами (таких как «Тариф») над признаками с меньшими диапазонами (такими как «Возраст») в процессе обучения, помогая всем моделям работать оптимально.                                                 |

### 4. Основные моменты кода

Давайте заглянем под капот и посмотрим на ключевые части кода, которые воплотили все это в жизнь.

*   **Загрузка данных:** Этот фрагмент кода является нашей отправной точкой, он считывает необработанную информацию о пассажирах «Титаника» в нашу систему.

    ```python
    def load_data():
        train_df = pd.read_csv(TRAIN_FILE)
        test_df = pd.read_csv(TEST_FILE)
        return train_df, test_df
    ```
    *Интерпретация:* Эта функция просто извлекает обучающие и тестовые наборы данных из мест их хранения, делая их готовыми к обработке.

*   **Очистка и подготовка признаков:** Эта важная функция преобразует необработанные данные в формат, подходящий для моделей машинного обучения. Она занимается удалением ненужных столбцов и заполнением пропущенных значений.

    ```python
    def transform_data(dataset: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        # ... (column dropping and imputation logic) ...
        cols_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
        for col in cols_to_drop:
            if col in features.columns:
                features = features.drop(columns=[col])

        numeric_cols = features.select_dtypes(include=np.number).columns
        categorical_cols = features.select_dtypes(exclude=np.number).columns

        if len(numeric_cols) > 0:
            features_numeric_copy = features[numeric_cols].copy()
            numeric_imputer = SimpleImputer(strategy='mean')
            features[numeric_cols] = numeric_imputer.fit_transform(features_numeric_copy)

        if len(categorical_cols) > 0:
            features_categorical_copy = features[categorical_cols].copy()
            categorical_imputer = SimpleImputer(strategy='most_frequent')
            features[categorical_cols] = categorical_imputer.fit_transform(features_categorical_copy)

        return features.values, target
    ```
    *Интерпретация:* Этот сегмент кода гарантирует, что ненужные точки данных отбрасываются, а все числовые пробелы заполняются средними значениями, в то время как категориальные пробелы заполняются наиболее распространенной категорией. Затем данные преобразуются в числовой массив, который является языком, понятным нашим моделям.

*   **Обучение и оценка модели AutoML:** Здесь происходит волшебство – модель учится на обучающих данных, а затем проверяется ее производительность.

    ```python
    def create_model():
        train_df, test_df = load_data()
        test_passenger_ids = test_df['PassengerId'].copy() # Preserve IDs for submission

        # Data splitting for training and evaluation
        train_data, eval_test_data = train_test_split(
            train_df,
            test_size=EVAL_SET_SIZE,
            random_state=SEED,
            stratify=train_df['Survived']
        )

        train_features, train_target = transform_data(train_data)
        eval_test_features, eval_test_target = transform_data(eval_test_data)
        test_features, _ = transform_data(test_df)

        # Train the model
        model = train_model(train_features, train_target.flatten())

        # Evaluate the model
        model_performance = evaluate_model(model, eval_test_features, eval_test_target.flatten())

        # Make predictions on the unseen test data
        predictions:np.ndarray = automl_predict(model, test_features)
        # ... (submission file creation) ...
        return model_performance
    ```
    *Интерпретация:* Эта центральная функция организует весь процесс: загрузку данных, их разделение на обучающие и проверочные наборы, преобразование, обучение сложного конвейера AutoML, оценку его навыков и, наконец, генерацию предсказаний для реальных тестовых данных.

*   **Создание файла отправки:** Последний шаг — форматирование наших предсказаний в требуемый файл отправки.

    ```python
    # ... (inside create_model function, after predictions are made) ...
    predictions = predictions.astype(int) # Ensure predictions are 0s or 1s

    output = pd.DataFrame({
        'PassengerId': test_passenger_ids,
        'Survived': predictions
    })
    output.to_csv(SUBMISSION_PATH, index=False)
    ```
    *Интерпретация:* Этот код берет бинарные предсказания нашей модели (0 для погибших, 1 для выживших) и сопоставляет их с исходным `PassengerId` из тестового набора, создавая аккуратно отформатированный CSV-файл, готовый для отправки на соревнование Kaggle.

### 5. Метрики

Производительность нашей модели измерялась с помощью **точности**.

*   **Точность:** `0.788`
*   *Что это значит:* Точность показывает, как часто модель дает правильные предсказания. Точность в 0.788 (или 78.8%) означает, что наша модель правильно предсказала статус выживаемости примерно для 78.8% пассажиров в проверочном наборе данных.

### 6. Выводы

Эта модель AutoML, использующая сложный ансамбль алгоритмов машинного обучения и надежную подготовку данных, продемонстрировала высокую способность предсказывать выживаемость на «Титанике». С точностью почти 79%, она эффективно изучила закономерности из исторических данных, чтобы делать обоснованные предсказания о том, кто выжил в трагической катастрофе. Это демонстрирует силу автоматизированного машинного обучения в решении реальных задач классификации со значительным успехом.